In [2]:
import cv2
from typing import NamedTuple, Tuple, List, Iterable
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy.ndimage.filters as filters
from itertools import chain

In [13]:
vid = cv2.VideoCapture('/home/sh3mm/Desktop/IFT6757/image/videos/video_1.avi')
fps = vid.get(cv2.CAP_PROP_FPS)
print(fps)

ret, last_img = vid.read()
last_img = cv2.cvtColor(last_img, cv2.COLOR_BGR2GRAY)

buffer = [last_img]
diff_buffer = []

while vid.isOpened():
    ret, curr_img = vid.read()
    if not ret:
        break

    curr_img = cv2.cvtColor(curr_img, cv2.COLOR_BGR2GRAY)

    diff = np.abs(curr_img.astype(int) - buffer[-1])
    diff[diff < 50] = 0

    buffer.append(curr_img)
    diff_buffer.append(diff)

15.0


In [4]:
class Point(NamedTuple):
    coords: Tuple[int, int]
    last_seen: int = 0
    weight: int = 1

    def __add__(self, other: 'Point'):
        new_weight = other.weight + self.weight
        new_coords = (
            (other.coords[0] * other.weight + self.coords[0] * self.weight) // new_weight,
            (other.coords[1] * other.weight + self.coords[1] * self.weight) // new_weight
        )
        new_last_seen = min(other.last_seen, self.last_seen)

        return Point(new_coords, new_last_seen, new_weight)

    @staticmethod
    def batch(batch: Iterable[Tuple[int, int]]) -> List['Point']:
        return [Point(b) for b in batch]


def group_point(points: List[Point], size):
    group_nb = 0
    groups = []
    for p1 in points:
        i1, j1 = p1.coords
        for p2, nb in chain(*groups):
            i2, j2 = p2.coords
            if abs(i2 - i1) < size and abs(j2 - j1) < size:
                groups[nb].append((Point((i1, j1)), nb))
                break
        else:
            groups.append([(Point((i1, j1)), group_nb)])
            group_nb += 1

    return [sum((e[0] for e in group[1:]), start=group[0][0]) for group in groups]


def get_maxes(base_img):
    size = 9
    threshold = 120

    gaussian_blur = cv2.GaussianBlur(src=np.uint8(base_img), ksize=(size, size), sigmaX=3, sigmaY=3)
    data_max = filters.maximum_filter(gaussian_blur, size)

    maxima = (gaussian_blur == data_max)

    diff = ((data_max - 0) > threshold)
    maxima[diff == 0] = 0

    return group_point(Point.batch(zip(*maxima.nonzero())), 2)

In [14]:
print(get_maxes(diff_buffer[26]))

[]


In [16]:
coords = set(chain(*[get_maxes(i) for i in diff_buffer]))
coords = group_point(coords, 6)
coords
#img_y, img_x = list(zip(*coords))
#px.imshow(last_img, binary_string=True)\
#    .add_trace(go.Scatter(x=img_x,y=img_y, mode='markers', marker={'color': 'green'}))\
#    .show()

[Point(coords=(145, 330), last_seen=0, weight=4),
 Point(coords=(146, 319), last_seen=0, weight=2),
 Point(coords=(176, 322), last_seen=0, weight=2)]

In [29]:
def get_sub(img_buffer: Iterable[np.ndarray], center: Point, radius: int) -> np.ndarray:
    return np.array(img_buffer)[
        :,
        center.coords[0] - radius: center.coords[0] + radius + 1,
        center.coords[1] - radius: center.coords[1] + radius + 1
    ]


def get_frequency(img_buffer: np.ndarray, frame_rate=30, tolerance=.35) -> Tuple[float, np.ndarray]:
    histogram = np.sum(img_buffer, axis=(1, 2), dtype=int)
    first_diff = np.clip(np.diff(histogram), 0, np.inf)
    second_diff = np.clip(np.diff(first_diff), 0, np.inf)

    if not np.sum(second_diff > 5e3):
        return -1, (second_diff > 5e3).astype(int)

    spikes = (second_diff / np.max(second_diff)) > tolerance
    spike_freq = frame_rate / np.median(np.diff(spikes.nonzero()))

    return spike_freq, spikes.astype(int)


In [30]:
fr0, changes0 = get_frequency(get_sub(buffer, coords[0], 10)[:25])
fr1, changes1 = get_frequency(get_sub(buffer, coords[1], 10))
fr2, changes2 = get_frequency(get_sub(buffer, coords[2], 10))

print(fr0, fr1, fr2)
print(coords)
px.bar(changes0).show()
px.bar(changes1).show()
px.bar(changes2).show()

-1 1.0 1.0
[Point(coords=(145, 330), last_seen=0, weight=4), Point(coords=(146, 319), last_seen=0, weight=2), Point(coords=(176, 322), last_seen=0, weight=2)]
